For the top `n` people in both papers, get data about the distribution of article types in which they appear.

- Get lists of top n people per paper
- Merge and dedupe those
- Use resulting list to filter the annualized data
- Merge all of the relevant rows into a new data frame
- Groupby name (and year?) and count article type

In [37]:
import os
import pandas as pd
from multiprocessing import Pool

In [4]:
NAMES = '/oak/stanford/groups/malgeehe/celebs/chicago_results/names_annual'

In [9]:
TOP = '/oak/stanford/groups/malgeehe/celebs/chicago_results/chicago_name_paper.csv'

In [32]:
META = '/oak/stanford/groups/malgeehe/celebs/chicago_results/chicago_1919-1939_meta.csv'

In [10]:
names = pd.read_csv(TOP)

In [11]:
names.head()

,person,paper,n_mentions
0,N. Clark,Chicago Daily Tribune,10358
1,- Cago,Chicago Daily Tribune,8054
2,N. Y.,Chicago Daily Tribune,6072
3,Van Buren,Chicago Daily Tribune,5497
4,W. Madison,Chicago Daily Tribune,5250


In [12]:
names.shape[0]

2761876

Get top n people per paper:

In [13]:
papers = names['paper'].unique()

In [38]:
N = 1000

In [39]:
L = []
for paper in papers:
    sub = names[names['paper'] == paper].sort_values('n_mentions', ascending = False)
    L.append(list(sub['person'][:N]))

In [40]:
len(L[0]), len(L[1])

(1000, 1000)

In [41]:
combined = set(L[0] + L[1])

In [42]:
len(combined)

1908

In [43]:
combined = list(combined)

Filtering the annualized data

In [29]:
names_files = [os.path.join(NAMES,x) for x in os.listdir(NAMES) if '._' not in x]

In [57]:
def get_data(path):
    data = pd.read_csv(path)
    sub = data[data['person'].isin(combined)]
    return sub[['person','paper','year','doc_type']]

In [58]:
with Pool() as p:
    data = p.map(get_data, names_files)

In [59]:
len(data)

21

In [60]:
df = pd.concat(data)

In [69]:
g = df.groupby(['person','year','doc_type']).count()

In [70]:
g.head()

paper
person year doc_type                     
, Jr.  1919 Advertisement              69
            Feature                   144
            Obituary                  125
       1920 Feature                   140
            Front Page/Cover Story      2